In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset
dataset=dataset.replace({True: 1,False: 0})
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Maheshwaran\anaconda3\Lib\site-packages\sklearn\tree\_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [12]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [13]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'auto', 'splitter': 'best'}: 0.9031880225910075


In [14]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 6 43]]


In [15]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.92        85
           1       0.86      0.88      0.87        49

    accuracy                           0.90       134
   macro avg       0.89      0.90      0.90       134
weighted avg       0.90      0.90      0.90       134



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.8975990396158462

In [17]:
table=pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015625,0.000002,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.765716,0.849057,0.833323,0.868632,0.905069,0.844359,0.046051,4
1,0.006250,0.007655,0.006250,0.007655,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.832255,0.832918,0.815056,0.793565,0.850543,0.824867,0.019260,12
2,0.000000,0.000000,0.006250,0.007654,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.813511,0.851527,0.906166,0.844370,0.844068,0.035750,5
3,0.000000,0.000000,0.003125,0.006249,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.849057,0.777291,0.887907,0.808775,0.834034,0.037835,8
4,0.000000,0.000000,0.006249,0.007653,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.804764,0.811321,0.814409,0.870362,0.906166,0.841404,0.040022,6
5,0.000000,0.000000,0.003125,0.006250,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.843811,0.866968,0.832483,0.793565,0.885265,0.844418,0.031324,3
6,0.003125,0.006250,0.006250,0.007655,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.847141,0.849057,0.832483,0.851527,0.905069,0.857055,0.024908,1
7,0.000000,0.000000,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.799620,0.813511,0.741078,0.870362,0.943041,0.833523,0.068480,9
8,0.003125,0.006251,0.003125,0.006251,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.759544,0.886792,0.833323,0.831098,0.844370,0.831026,0.040995,10
9,0.006249,0.007653,0.000000,0.000000,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.905342,0.811321,0.814409,0.868632,0.805463,0.841033,0.039381,7


In [19]:
Age=float(input("Age:"))
EstimatedSalary=float(input("Estimated_Salary:"))
Gender_Male=int(input("Sex Male 1 or 0:"))

Age: 40
Estimated_Salary: 45
Sex Male 1 or 0: 0


In [20]:
Future_Prediction=grid.predict([[Age,EstimatedSalary,Gender_Male]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
